In [ ]:
import gradio as gr

from src.engine import GradioEngine
import src.fe_variables as app_metadata

# Create GradioEngine Object
session = GradioEngine()

# Gradio app configuration

Add in some extra fields
- Testament
- Cross encoder or regular search
- Number of results
- Examples

In [ ]:

# ====== #
# Tab1: Verse Similarity
# ====== #

with gr.Blocks(title=app_metadata.TAB1_PAGE_NAME, theme=gr.themes.Glass()) as AskVerse:
    # Instructions
    instructions = gr.Markdown(value=app_metadata.TAB1_INSTRUCTIONS)
    # Input
    tab1_question = gr.Textbox(label="Question:", placeholder="Ex. Who is God?")
    # Buttons
    tab1_run_btn = gr.Button("Run")
    # Output
    tab1_response = gr.DataFrame(label="Answer:", interactive=False)
    # More Buttons & Export Data
    tab1_export_btn = gr.Button("Download Results as csv")
    tab1_csv = gr.File(interactive=False, visible=False)
    tab1_clear = gr.ClearButton([tab1_question, tab1_response, tab1_csv])

    # Execute
    tab1_run_btn.click(session.tab1_engine, tab1_question, tab1_response)
    tab1_export_btn.click(session.export_csv, tab1_response, tab1_csv)


# ====== #
# Tab: FAQ
# ====== #
with gr.Blocks(title=app_metadata.TAB4_PAGE_NAME, theme=gr.themes.Glass()) as FAQ:
    # Instructions
    instructions = gr.Markdown(value=app_metadata.TAB4_INSTRUCTIONS)


# ====== #
# Put it together
# ====== #
demo = gr.TabbedInterface([AskVerse, FAQ], [app_metadata.TAB1_PAGE_NAME, app_metadata.TAB4_PAGE_NAME], title=app_metadata.APP_NAME)

if __name__ == "__main__":
    demo.launch()

# Testing

In [1]:
from src.engine import GradioEngine
# Create GradioEngine Object
session = GradioEngine()

In [5]:
print(session.verse_data['embeddings'])

0      [0.02933773584663868, 0.09763185679912567, 0.0...
1      [-0.030829861760139465, 0.06714971363544464, 0...
2      [-0.042698148638010025, 0.11107104271650314, -...
3      [0.008858589455485344, 0.11572469025850296, -0...
4      [0.02279425598680973, 0.08372653275728226, 0.0...
                             ...                        
874    [-0.0199922826141119, 0.05543883144855499, -0....
875    [-0.046769507229328156, 0.04644876718521118, 0...
876    [-0.0060417004860937595, 0.027661064639687538,...
877    [-0.04907679185271263, 0.06318483501672745, 0....
878    [-0.013146620243787766, 0.05486626550555229, 0...
Name: embeddings, Length: 879, dtype: object


In [3]:
# Testing tab1
tab2_data = session.tab1_engine(question="Who is God?", run_cross_encoder="No", testament="New")
print(tab2_data)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U8439')) -> None

In [ ]:
session.verse_data